<a href="https://colab.research.google.com/github/opsabarsec/NLP--film-genres-from-synopsis/blob/main/radix_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic assignment of genres from movie synopsis using supervised machine learning

## 1. Import libraries and load data

In [1]:
#packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# NLP libraries

from textblob import TextBlob, Word
import nltk
import re
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Deep learning libraries

from keras.models import Model
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Input, LSTM, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer

In [2]:
# load data
train= pd.read_csv('./data/train.csv')
test=pd.read_csv('./data/test.csv')

## 2. DATA PREPARATION 

In [3]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# function for text cleaning 
def preprocess_text(text):
    text = text.lower() # lowercase
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text) #line breaks
    #text = re.sub(r"\'\xa0", " ", text) # xa0 Unicode representing spaces
    #text = re.sub('\s+', ' ', text) # one or more whitespace characters
    text = text.strip(' ') # spaces
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    #lemmatize and remove stopwords
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text) 
        
    return text

train['clean_plot'] = train['synopsis'].apply(lambda x: preprocess_text(x))
test['clean_plot'] = test['synopsis'].apply(lambda x: preprocess_text(x))

In [5]:
def lemma(text): # Lemmatization of cleaned body
        sent = TextBlob(text)
        tag_dict = {"J": 'a', 
                    "N": 'n', 
                    "V": 'v', 
                    "R": 'r'}
        words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
        lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
        seperator=' '
        lemma = seperator.join(lemmatized_list) 
        return lemma

In [6]:
train['lemma'] = train['clean_plot'].apply(lambda x: lemma(x))
test['lemma'] = test['clean_plot'].apply(lambda x: lemma(x))

## 3. Variables preparation 

In [7]:

X = train['lemma']
X_test = test['lemma']    

### 3.1 Target variable one hot encoding

In [8]:
#apply the onehot transformation for the genres vector
y = train['genres']
one_hot = MultiLabelBinarizer() # encoder for the  tags 
y_onehot = one_hot.fit_transform(y.str.split(' ')) 
y_bin = pd.DataFrame(y_onehot, columns=one_hot.classes_ ) # transform it to Pandas object

In [9]:
# tokenize
max_features = 5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))
list_tokenized_train = tokenizer.texts_to_sequences(X)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)

In [10]:
#fix max comment lenght
maxlen = 100
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

## 4.The Model

In [11]:
#initialize parameters
inp = Input(shape=(maxlen, )) #maxlen defined earlier
embed_size = 128

In [12]:
# Neural network backbone
x = Embedding(max_features, embed_size)(inp)

x = LSTM(64, return_sequences=True,name='lstm_layer')(x)

x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(len(y_bin.columns), activation="softmax")(x)

In [13]:
# build the model
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [23]:
# train the model
batch_size = 16
epochs = 3

hist = model.fit(X_t,y_onehot, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/3
2055/2055 [==============================] - 74s 36ms/step - loss: 0.1526 - accuracy: 0.5809 - val_loss: 0.2178 - val_accuracy: 0.4614
Epoch 2/3
2055/2055 [==============================] - 61s 30ms/step - loss: 0.1446 - accuracy: 0.5962 - val_loss: 0.2208 - val_accuracy: 0.4587
Epoch 3/3
2055/2055 [==============================] - 81s 39ms/step - loss: 0.1374 - accuracy: 0.6071 - val_loss: 0.2267 - val_accuracy: 0.4518


## 5.The prediction

In [15]:
y_pred = model.predict(X_te, batch_size=batch_size, verbose=1)
print(y_pred.shape)

329/329 [==============================] - 4s 11ms/step
(5250, 19)


In [16]:
df_probs_all = pd.DataFrame(y_pred,columns=y_bin.columns)

df_probs_all.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.153055,0.120783,0.063999,0.024218,0.044514,0.006373,0.025455,0.045988,0.066021,0.000368,0.071790,0.014119,0.002922,0.014618,0.007189,0.270556,0.048912,0.007685,0.011436
1,0.296934,0.080151,0.028525,0.006518,0.033059,0.011568,0.221694,0.041668,0.018811,0.000287,0.017067,0.013154,0.003310,0.008403,0.005361,0.142858,0.037881,0.022464,0.010285
2,0.000533,0.000298,0.000428,0.000230,0.003771,0.001269,0.962205,0.021533,0.000294,0.000025,0.000231,0.000244,0.003581,0.000646,0.001492,0.000237,0.000719,0.002181,0.000083
3,0.088475,0.039922,0.014833,0.005454,0.022711,0.002802,0.001868,0.029962,0.038622,0.000065,0.360627,0.002843,0.000127,0.013047,0.002047,0.291271,0.083592,0.000637,0.001094
4,0.060129,0.008724,0.001417,0.001615,0.077067,0.246958,0.005421,0.254630,0.003551,0.022893,0.022130,0.000629,0.001771,0.064526,0.023605,0.006738,0.188139,0.002471,0.007586


In [17]:
def top_5_predictions(df):
    N = 5
    cols = df.columns[:-1].tolist()
    a = df[cols].to_numpy().argsort()[:, :-N-1:-1]
    c = np.array(cols)[a]
    d = df[cols].to_numpy()[np.arange(a.shape[0])[:, None], a]
    df1 = pd.DataFrame(c).rename(columns=lambda x : f'max_{x+1}_col')

    predicted_genres = df1["max_1_col"] + ' ' + df1["max_2_col"]+ ' ' +df1["max_3_col"]+ ' ' + df1["max_4_col"]+ ' '+df1["max_5_col"]
    return predicted_genres

In [18]:
pred_gen = top_5_predictions(df_probs_all)

In [19]:
submission = pd.DataFrame(data= {'movie_id':test.movie_id,'predicted_genres':pred_gen})

In [20]:
submission.head()

,movie_id,predicted_genres
0,10827,Sci-Fi Action Adventure Horror Fantasy
1,51768,Action Documentary Sci-Fi Adventure Drama
2,36064,Documentary Drama Comedy Musical War
3,33763,Horror Sci-Fi Action Thriller Adventure
4,1146,Drama Crime Thriller Comedy Mystery


In [21]:
submission.to_csv('submission.csv',index=False)